# Install and Import 

In [1]:
# !pip install tensorflow==2.15.0
# !pip install gym
# !pip install keras
# !pip install keras-rl2
# %pip install scikit-learn

In [2]:
# %pip install ipykernel
# %pip install --upgrade nbformat

In [3]:
import numpy as np
import pandas as pd
from gym import Env
from gym import spaces
import math
from gym.spaces import Discrete, Box
import numpy as np
import random
import logging

In [4]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, BatchNormalization, Dropout, Conv1D, MaxPooling1D
# from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers.legacy import Adam

In [5]:
from keras import __version__
tf.keras.__version__ = __version__
from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.policy import LinearAnnealedPolicy, EpsGreedyQPolicy
from rl.memory import SequentialMemory

In [6]:
from model_config import Path
import os
import pandas as pd

# Open and Preprocessing Data

In [7]:
model_num = 4
data_num = 1

In [8]:
# Загрузка данных
df = pd.read_csv(Path["dataset"](model_num, data_num))

In [9]:
zero_close_prices = df[df['Close'] == 0]
print(f"Number of zero 'Close' prices after scaling: {len(zero_close_prices)}")

Number of zero 'Close' prices after scaling: 0


In [10]:
unique_values = df['Asset_ID_encoded'].unique()
unique_values

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25], dtype=int64)

In [11]:
value_counts = df['Asset_ID_encoded'].value_counts()
value_counts

Asset_ID_encoded
1     4412
23    4412
18    4412
0     4411
5     4411
16    4410
15    4408
19    4399
25    4395
4     4394
8     4354
13    4352
20    4339
3     4321
14    4313
6     4274
9     4241
21    4117
22    4095
11    3253
24    3242
10    3222
7     2957
17    2760
2     2574
12    1487
Name: count, dtype: int64

In [12]:
# Разделение данных временного ряда на тренировочную и тестовую выборки.
def train_test_split_time_series(df, train_size=0.8):
    split_index = int(len(df) * train_size)
    train_df = df.iloc[:split_index].reset_index(drop=True)
    test_df = df.iloc[split_index:].reset_index(drop=True)
    return train_df, test_df

In [13]:
# Выполнение разделения
train_df, test_df = train_test_split_time_series(df, train_size=0.8)
print(f"Training data shape: {train_df.shape}")
print(f"Testing data shape: {test_df.shape}")

Training data shape: (81572, 25)
Testing data shape: (20393, 25)


# Game Rule

### Пробуем вместо сброса баланса просто перезапускать модель с начала следующего токена в данных, также после смены токена просто продаем активы и не сбрасываем баласн.

### Скорее всего тут ошибка в ресете, если нет то фигная получилась

расскажи мне подробнее про Непрерывное Пространство для действий про которое ты писал: self.action_space = spaces.Поле(low=-1, high=1, shape=(1,), dtype=np.float32)
 как я могу внедрить его в свой проект, хотя ради тестирования.

In [14]:
window_length = 100
nb_steps = 80000

In [15]:
import decimal
decimal.getcontext().prec = 28  # Increase precision

In [16]:
# Define the trading environment
class TradingEnv(Env):
    """
    Environment for training an agent to trade on the exchange using the DQN algorithm.
    """
    metadata = {'render.modes': ['human']}

    def __init__(self, df):
        super(TradingEnv, self).__init__()

        # Save data and initialize parameters
        self.df = df.reset_index(drop=True)
        self.total_steps = len(self.df) - 1
        self.current_step = 0

        # Find indices where a new asset starts
        self.asset_start_indices = self._find_asset_start_indices()

        # Define action space: 0 - Hold, 1 - Buy 100%, 2 - Buy 50%, 3 - Sell 100%, 4 - Sell 50%
        self.action_space = spaces.Discrete(5)

        # Define observation space
        # Add an additional feature - number of shares held
        num_features = len(self.df.columns)
        self.observation_space = spaces.Box(
            low=-np.inf, high=np.inf, shape=(num_features + 1,), dtype=np.float32)

        # Initialize trading parameters
        self.fee_cost = 0.001
        self.initial_balance = 1000  # starting balance
        self.balance = self.initial_balance
        self.net_worth = self.initial_balance
        self.prev_net_worth = self.net_worth

        # self.now_token = df.loc[self.current_step, 'Asset_ID_encoded']
        self.now_token = -1 
        self.prev_token = self.now_token

        self.shares_held = 0
        self.total_shares_sold = 0
        self.total_sales_value = 0

        self.hist = {
            "current_step": [],
            'balance': [],
            'net_worth': [],
            'shares_held': [],
            "token": [],
            "current_price": [],
            "reward": [],
            "action": [],
            'total_shares_sold': [],
            'total_sales_value': [],
        }

        # Set up logging to a file
        self.logger = logging.getLogger(__name__)
        self.logger.setLevel(logging.INFO)

        # Remove any existing handlers
        if self.logger.hasHandlers():
            self.logger.handlers.clear()

        # Create a file handler
        file_handler = logging.FileHandler(Path["train_log"](model_num, data_num))
        file_handler.setLevel(logging.INFO)

        # Create a logging format
        formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
        file_handler.setFormatter(formatter)

        # Add the file handler to the logger
        self.logger.addHandler(file_handler)
        logging.disable(logging.CRITICAL)


    def _find_asset_start_indices(self):
        """
        Find the indices in the DataFrame where a new asset starts.
        """
        asset_ids = self.df['Asset_ID_encoded']
        start_indices = [0]
        for i in range(1, len(asset_ids)):
            if asset_ids[i] != asset_ids[i - 1]:
                start_indices.append(i)
        return start_indices


    def reset(self):
        """
        Reset the environment to the initial state.
        """
        self.logger.info("Environment reset")

        self.balance = self.initial_balance
        self.net_worth = self.initial_balance
        self.prev_net_worth = self.net_worth
        self.shares_held = 0
        self.cost_basis = 0
        self.total_shares_sold = 0
        self.total_sales_value = 0

        # Move to the next asset's starting index
        self.now_token += 1
        if self.now_token >= len(self.asset_start_indices):
            self.now_token = 0  # Loop back to the first asset

        self.current_step = self.asset_start_indices[self.now_token]
        print(self.current_step)
        self.now_token = self.df.loc[self.current_step, 'Asset_ID_encoded']
        self.prev_token = self.now_token

        self.logger.info(f"Starting new episode with token {self.now_token} at step {self.current_step}")

        self.hist = {
            "current_step": [],
            'balance': [],
            'net_worth': [],
            'shares_held': [],
            "token": [],
            "current_price": [],
            "reward": [],
            "action": [],
            'total_shares_sold': [],
            'total_sales_value': [],
        }
        return self._next_observation()


    def _next_observation(self):
        """
        Get the observation for the current step.
        """
        # Get data for the current step
        frame = self.df.loc[self.current_step]
        # Convert data to numpy array and add number of shares held
        obs = np.append(frame.values, [self.shares_held])
        return obs.astype(np.float32)


    def step(self, action):
        """
        Execute the action and update the environment state.
        """
        self._take_action(action)

        # Update current token
        self.prev_token = self.now_token
        self.now_token = self.df.loc[self.current_step, 'Asset_ID_encoded']
        

        if self.now_token != self.prev_token:
            self.logger.info(f"Token change at step {self.current_step}: {self.prev_token} -> {self.now_token}")
            self._sell_all_tokens()
            reward = 0 
            # Пробуем начислять новый капитал при смене токена
            self.logger.info("Reset after token change")
        else:
            # Calculate reward as change in net worth
            # Avoid division by zero
            if self.prev_net_worth != 0:
                if self.net_worth <= self.initial_balance * 0.5:
                    done = True
                    
                reward = (self.net_worth - self.prev_net_worth) / self.prev_net_worth * 100
            else:
                reward = 0


        self.prev_net_worth = self.net_worth
        self.current_step += 1

        # Check if end of data is reached
        done = self.current_step >= self.total_steps

        obs = self._next_observation()

        # Logging
        self.logger.info(f"Step: {self.current_step}, Action taken: {action}, Reward: {reward}")
        self.logger.info(f"Net worth: {self.net_worth}, Balance: {self.balance}, ")

        self.hist["current_step"].append(self.current_step)
        self.hist["balance"].append(self.balance)
        self.hist["net_worth"].append(self.net_worth)
        self.hist["shares_held"].append(self.shares_held)
        self.hist["token"].append(self.now_token)
        self.hist["current_price"].append(self.current_price)
        self.hist["reward"].append(reward)
        self.hist["action"].append(action)
        self.hist["total_shares_sold"].append(self.total_shares_sold)
        self.hist["total_sales_value"].append(self.total_sales_value)

        return obs, reward, done, {}


    def _sell_all_tokens(self):
        self.current_price = self.df.loc[self.current_step - 1, 'Close']
        shares_to_sell = self.shares_held
        if shares_to_sell > 0:
            # Determine total sale amount
            total_sale = shares_to_sell * self.current_price
            transaction_cost = total_sale * self.fee_cost
            total_sale -= transaction_cost  # Corrected: subtract transaction cost

            self.balance += total_sale
            self.shares_held = 0
            self.total_shares_sold += shares_to_sell  # Corrected: use shares_to_sell
            self.total_sales_value += total_sale

            # Update net worth
            self.net_worth = self.balance

            self.logger.info(f"Sold all shares of token {self.prev_token} at price {self.current_price}")
            self.logger.info(f"Total sale: {total_sale}, Transaction cost: {transaction_cost}")
        else:
            self.logger.info("No shares to sell.")


    def _take_action(self, action):
        """
        Apply the action (buy, sell, hold) to the current state.
        """
        self.current_price = self.df.loc[self.current_step, 'Close']

        if action == 1:  # Buy 100%
            # Determine the maximum number of shares that can be bought
            max_possible_shares = int(self.balance / (self.current_price * (1 + self.fee_cost)))
            shares_to_buy = int(math.floor(max_possible_shares))
            if shares_to_buy > 0:
                total_cost = shares_to_buy * self.current_price
                transaction_cost = total_cost * self.fee_cost
                total_cost += transaction_cost

                self.balance -= total_cost
                self.shares_held += shares_to_buy

                self.logger.info(f"Bought {shares_to_buy} shares at price {self.current_price}")
                self.logger.info(f"Total cost: {total_cost}, Transaction cost: {transaction_cost}")
            else:
                self.logger.info("Not enough balance to buy.")


        elif action == 2:  # Buy 50%
            # Determine number of shares to buy
            amount_to_spend = self.balance * 0.5
            max_possible_shares = int(amount_to_spend / (self.current_price * (1 + self.fee_cost)))
            shares_to_buy = int(math.floor(max_possible_shares))
            if shares_to_buy > 0:
                total_cost = shares_to_buy * self.current_price
                transaction_cost = total_cost * self.fee_cost
                total_cost += transaction_cost

                self.balance -= total_cost
                self.shares_held += shares_to_buy

                self.logger.info(f"Bought {shares_to_buy} shares at price {self.current_price}")
                self.logger.info(f"Total cost: {total_cost}, Transaction cost: {transaction_cost}")
            else:
                self.logger.info("Not enough balance to buy.")


        elif action == 3 and self.shares_held > 0:  # Sell 100%
            # Determine number of shares to sell
            shares_to_sell = self.shares_held
            total_sale = shares_to_sell * self.current_price
            transaction_cost = total_sale * self.fee_cost
            total_sale -= transaction_cost  # Corrected: subtract transaction cost

            self.balance += total_sale
            self.shares_held -= shares_to_sell
            self.total_shares_sold += shares_to_sell
            self.total_sales_value += total_sale

            self.logger.info(f"Sold {shares_to_sell} shares at price {self.current_price}")
            self.logger.info(f"Total sale: {total_sale}, Transaction cost: {transaction_cost}")


        elif action == 4 and self.shares_held > 0:  # Sell 50%
            # Determine number of shares to sell
            shares_to_sell = int(math.floor(self.shares_held * 0.5))
            total_sale = shares_to_sell * self.current_price
            transaction_cost = total_sale * self.fee_cost
            total_sale -= transaction_cost  # Corrected: subtract transaction cost

            self.balance += total_sale
            self.shares_held -= shares_to_sell
            self.total_shares_sold += shares_to_sell
            self.total_sales_value += total_sale

            self.logger.info(f"Sold {shares_to_sell} shares at price {self.current_price}")
            self.logger.info(f"Total sale: {total_sale}, Transaction cost: {transaction_cost}")


        else:  # Hold or invalid action
            self.logger.info("Holding position.")

        # Update net worth
        self.net_worth = self.balance + self.shares_held * self.current_price


    def render(self, mode='human', close=False):
        """
        Output the current state of the environment.
        """
        profit = self.net_worth - self.initial_balance
        print(f'Step: {self.current_step}')
        print(f'Balance: {self.balance:.2f}')
        print(f'Shares held: {self.shares_held}')
        print(f'Net worth: {self.net_worth:.2f}')
        print(f'Profit: {profit:.2f}')


In [17]:
# Создание экземпляров среды для тренировки и тестирования
train_env = TradingEnv(train_df)
test_env = TradingEnv(test_df)
nb_actions = train_env.action_space.n

# Model Config

In [18]:
# Определение архитектуры нейронной сети для DQN
model = Sequential()
model.add(Flatten(input_shape=(window_length,) + train_env.observation_space.shape))
model.add(Dense(128, activation='relu'))  # Увеличено количество нейронов
model.add(Dropout(0.2))  # Добавлен Dropout для регуляризации
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(32, activation='relu'))
model.add(Dense(nb_actions, activation='linear'))

# print(model.summary())

In [19]:
# Настройка памяти и политики для DQN
memory = SequentialMemory(limit=500000, window_length=window_length)
policy = LinearAnnealedPolicy(
    EpsGreedyQPolicy(),
    attr='eps',
    value_max=1.0,
    value_min=0.1,
    value_test=0.05,
    nb_steps=nb_steps
)  # Использование линейного уменьшения epsilon

In [20]:
# Создание агента DQN с улучшенными параметрами
dqn = DQNAgent(
    model=model,
    nb_actions=nb_actions,
    memory=memory,
    nb_steps_warmup= (nb_steps //10),  # Увеличен период разогрева
    target_model_update=1000,  # Частота обновления целевой модели
    policy=policy,
    enable_double_dqn=True  # Включение Double DQN для снижения переоценки Q-значений
)

In [21]:
optimizer = Adam(learning_rate=1e-4)  # Уменьшение скорости обучения
dqn.compile(optimizer, metrics=['mae'])

# Train Model

In [22]:
train_history = dqn.fit(
    train_env,
    nb_steps=nb_steps,  
    visualize=False,
    verbose=1,
    log_interval=10000  
)

Training for 80000 steps ...
0
Interval 1 (0 steps performed)
   46/10000 [..............................] - ETA: 11s - reward: 0.1211      

d:\PythonScripts\trader_game\.venv\Lib\site-packages\keras\src\engine\training_v1.py:2359: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


10000/10000 [==============================] - 48s 5ms/step - reward: 0.0182
Interval 2 (10000 steps performed)
10000/10000 [==============================] - 215s 22ms/step - reward: -0.0233
Interval 3 (20000 steps performed)
10000/10000 [==============================] - 234s 23ms/step - reward: -0.0065
Interval 4 (30000 steps performed)
10000/10000 [==============================] - 246s 25ms/step - reward: -0.0299
Interval 5 (40000 steps performed)
10000/10000 [==============================] - 257s 26ms/step - reward: 0.2061
Interval 6 (50000 steps performed)
10000/10000 [==============================] - 273s 27ms/step - reward: 0.0314
Interval 7 (60000 steps performed)
10000/10000 [==============================] - 287s 29ms/step - reward: 0.0369
Interval 8 (70000 steps performed)
10000/10000 [==============================] - 306s 31ms/step - reward: -0.0098
done, took 1867.476 seconds


In [23]:
train_hist = train_env.hist
print(len(train_hist["action"]))
train_hist_df = pd.DataFrame(train_hist)

80000


In [24]:
# Assume 'dqn' is your trained DQNAgent
dqn.save_weights(Path["save_model"](model_num, data_num), overwrite=True)

# Visualisaton

In [25]:
import plotly.express as px
import plotly.graph_objects as go

In [26]:
def plot_close_by_asset(df, asset_id):
    # Фильтрация данных по Asset_ID_encoded
    asset_data = df[df['Asset_ID_encoded'] == asset_id]
    
    # Построение графика Close к индексу DataFrame
    fig = px.line(asset_data, x=asset_data.index, y='Close', 
                  title=f'Close Price for Asset ID {asset_id}', 
                  labels={'index': 'Index', 'Close': 'Close Price'})
    
    # Показать график
    fig.show()

In [27]:
def plot_price_change_by_asset(df, asset_id):
    # Фильтрация данных по Asset_ID_encoded
    asset_data = df[df['Asset_ID_encoded'] == asset_id].copy()
    
    # Вычисление процентного изменения цены (Close)
    asset_data['Price_Change_Percent'] = asset_data['Close'].pct_change() * 100
    
    # Построение графика изменения цены в процентах
    fig = px.line(asset_data, x=asset_data.index, y='Price_Change_Percent', 
                  title=f'Price Change Percentage for Asset ID {asset_id}', 
                  labels={'index': 'Index', 'Price_Change_Percent': 'Price Change (%)'})
    
    # Показать график
    fig.show()

In [28]:
def plot_token_data(df, token):
    # Фильтрация данных по выбранному токену
    token_data = df[df['token'] == token]
    
    # Вычисление среднего значения net_worth для данного токена
    avg_net_worth = token_data['net_worth'].mean()

    # Создание графика
    fig = go.Figure()

    # Линия net_worth
    fig.add_trace(go.Scatter(x=token_data['current_step'], y=token_data['net_worth'], mode='lines', name='Net Worth'))

    # Горизонтальная линия для net_worth = 1000
    fig.add_hline(y=1000, line_color="green", name='Net Worth = 1000')

    # Горизонтальная линия для среднего значения net_worth
    fig.add_hline(y=avg_net_worth, line_color="red", name=f'Average Net Worth = {avg_net_worth:.2f}')

    # Настройка заголовков и осей
    fig.update_layout(title=f'Net Worth and Average for {token}',
                      xaxis_title='Current Step',
                      yaxis_title='Net Worth')

    # Показать график
    fig.show()

In [29]:
def plot_action_counts(df, token):
    # Фильтрация данных по токену
    token_data = df[df['token'] == token]
    
    # Подсчет количества каждого уникального действия для данного токена
    action_counts = token_data['action'].value_counts().reset_index()
    action_counts.columns = ['action', 'count']

    # Построение бар-чарта для отображения количества каждого действия
    fig = px.bar(action_counts, x='action', y='count', title=f'Count of Actions for {token}', labels={'action': 'Action', 'count': 'Count'})

    # Показать график
    fig.show()

In [30]:
def plot_relative_change_by_token(df, token):
    # Фильтрация данных по токену
    token_data = df[df['token'] == token].copy()

    # Вычисление относительного изменения для current_price и net_worth
    token_data['Price_Change_Percent'] = token_data['current_price'].pct_change() * 100
    token_data['NetWorth_Change_Percent'] = token_data['net_worth'].pct_change() * 100

    # Создание графика
    fig = go.Figure()

    # Линия для изменения current_price
    fig.add_trace(go.Scatter(x=token_data['current_step'], y=token_data['Price_Change_Percent'],
                             mode='lines', name='Current Price Change (%)'))

    # Линия для изменения net_worth
    fig.add_trace(go.Scatter(x=token_data['current_step'], y=token_data['NetWorth_Change_Percent'],
                             mode='lines', name='Net Worth Change (%)'))

    # Настройка заголовков и осей
    fig.update_layout(title=f'Relative Change of Current Price and Net Worth for {token}',
                      xaxis_title='Current Step',
                      yaxis_title='Change (%)')

    # Показать график
    fig.show()

In [31]:
token = 19

In [32]:
plot_close_by_asset(df= train_df, asset_id= token)

In [33]:
plot_price_change_by_asset(df= train_df, asset_id= token)

In [34]:
plot_token_data(df = train_hist_df, token = token)

In [35]:
plot_action_counts(df = train_hist_df, token = token)

In [36]:
plot_relative_change_by_token(df = train_hist_df, token = token)

In [37]:
train_hist_df

,current_step,balance,net_worth,shares_held,token,current_price,reward,action,total_shares_sold,total_sales_value
0,1,1.000000e+03,1.000000e+03,0,0,5.630000e-07,0.000000,4,0,0.000000e+00
1,2,3.853841e-07,9.990010e+02,1796764386,0,5.560000e-07,-0.099900,1,0,0.000000e+00
2,3,9.908221e+02,9.908221e+02,0,0,5.520000e-07,-0.818705,3,1796764386,9.908221e+02
3,4,5.246151e-07,9.898323e+02,1796428847,0,5.510000e-07,-0.099900,1,1796764386,9.908221e+02
4,5,5.246151e-07,9.682751e+02,1796428847,0,5.390000e-07,-2.177858,1,1796764386,9.908221e+02
...,...,...,...,...,...,...,...,...,...,...
79995,79996,1.220991e+09,1.220991e+09,0,20,9.219461e+00,0.000000,0,2185284945111315479,3.818308e+12
79996,79997,1.220991e+09,1.220991e+09,0,20,9.168637e+00,0.000000,0,2185284945111315479,3.818308e+12
79997,79998,1.220991e+09,1.220991e+09,0,20,9.152049e+00,0.000000,0,2185284945111315479,3.818308e+12
79998,79999,1.220991e+09,1.220991e+09,0,20,9.230273e+00,0.000000,0,2185284945111315479,3.818308e+12


# Test Model

In [38]:
# Тестирование модели на тестовых данных
train_test_history = dqn.test(
    train_env,
    nb_episodes=3,  # Увеличено количество тестовых эпизодов
    visualize=False
)

Testing for 3 episodes ...
0
Episode 1: reward: 0.000, steps: 81571
0
Episode 2: reward: 0.000, steps: 81571
0
Episode 3: reward: 0.000, steps: 81571


In [39]:
train_test_hist = train_env.hist
train_test_hist_df = pd.DataFrame(train_test_hist)
print(len(train_test_hist["action"]))

81571


In [40]:
# Тестирование модели на тестовых данных
test_history = dqn.test(
    test_env,
    nb_episodes=3,  # Увеличено количество тестовых эпизодов
    visualize=False
)

Testing for 3 episodes ...
0
Episode 1: reward: 0.000, steps: 20392
0
Episode 2: reward: 0.000, steps: 20392
0
Episode 3: reward: 0.000, steps: 20392


In [41]:
test_hist = test_env.hist
test_hist_df = pd.DataFrame(test_hist)
print(len(test_hist["action"]))

20392


In [42]:
test_hist_df

,current_step,balance,net_worth,shares_held,token,current_price,reward,action,total_shares_sold,total_sales_value
0,1,1000,1000.0,0,20,7.953037,0.0,0,0,0
1,2,1000,1000.0,0,20,8.005000,0.0,0,0,0
2,3,1000,1000.0,0,20,8.072071,0.0,0,0,0
3,4,1000,1000.0,0,20,8.035708,0.0,0,0,0
4,5,1000,1000.0,0,20,8.085484,0.0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
20387,20388,1000,1000.0,0,25,545.742011,0.0,0,0,0
20388,20389,1000,1000.0,0,25,543.036064,0.0,0,0,0
20389,20390,1000,1000.0,0,25,535.652202,0.0,0,0,0
20390,20391,1000,1000.0,0,25,533.535339,0.0,0,0,0


In [43]:
unique_values = test_hist_df['token'].unique()
unique_values

array([20, 21, 22, 23, 24, 25], dtype=int64)

In [48]:
token = 21

In [49]:
plot_token_data(df = test_hist_df, token = token)

In [50]:
plot_close_by_asset(df= test_df, asset_id= token)

In [51]:
plot_action_counts(df = test_hist_df, token = token)